# Tools 2 Final Project

Jen Lee
Connie Chang

In [1]:
import pandas as pd
import geopandas as gpd
import re
import numpy as np

## Data Introduction

|Date Compiled |County/City|File Nale   |Description |
|:-------------|:---------:|:----------:|-----------:|
|29 March, 2021|Denver     |listings.csv|Summary information and metrics for listings in Denver (good for visualisations).|
|29 March, 2021|Denver     |calendar.csv| Detailed Calendar Data for listings in Denver|
|29 March, 2021|Denver     |reviews.csv| Summary Review data and Listing ID (to facilitate time based analytics and visualisations linked to a listing).|
|29 March, 2021|Denver     |neighbourhoods.csv| Neighbourhood list for geo filter. Sourced from city or open source GIS files.|
|29 March, 2021|Denver     |neighbourhoods.geojson| GeoJSON file of neighbourhoods of the city.|


Import each of the csv files. Not sure which ones we'll use yet, but might as well import all of them.



In [2]:
# listings.csv
listings = pd.read_csv("./data/denver_03_29_2021/listings.csv")
listings.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,177,https://www.airbnb.com/rooms/177,20210329150448,2021-03-30,Tiny Home in the Heart of the City- ECO FRIENDLY,"160 sq ft + 80 sq ft loft for sleeping, Sleeps...","Quiet neighborhood next to park, creeks and bi...",https://a0.muscache.com/pictures/a1745ab2-b3a0...,615,https://www.airbnb.com/users/show/615,...,10.0,10.0,9.0,2016-BFN-0004968,t,2,2,0,0,1.40
1,360,https://www.airbnb.com/rooms/360,20210329150448,2021-03-30,Sit in the Peaceful Garden of the Chickadee Co...,Enjoy the famous Colorado weather and unplug i...,The cottage is located in the center of Lower ...,https://a0.muscache.com/pictures/monet/Select-...,666,https://www.airbnb.com/users/show/666,...,10.0,10.0,10.0,2017-BFN-0002177,f,2,2,0,0,4.34
2,364,https://www.airbnb.com/rooms/364,20210329150448,2021-03-31,Lodo / RiNo LOFT via airport train,"Modern 1,000 square foot loft in the heart of ...","Ten brewpubs within walking distance, two grea...",https://a0.muscache.com/pictures/11766413/a2c5...,783,https://www.airbnb.com/users/show/783,...,10.0,9.0,9.0,NaN,f,1,1,0,0,0.60
3,590,https://www.airbnb.com/rooms/590,20210329150448,2021-04-02,Comfortable - and a great value!,"Large guest room in my home, where I also live...",I love the diversity of my neighborhood and it...,https://a0.muscache.com/pictures/110931/30991c...,933,https://www.airbnb.com/users/show/933,...,10.0,10.0,10.0,2021-BFN-0000578,f,2,0,2,0,4.14
4,592,https://www.airbnb.com/rooms/592,20210329150448,2021-03-31,private,This room is in the basement. It does not hav...,NaN,https://a0.muscache.com/pictures/110942/171dd0...,933,https://www.airbnb.com/users/show/933,...,10.0,10.0,10.0,2021-BFN-0000578,f,2,0,2,0,1.06


In [3]:
# calendar.csv
calendar = pd.read_csv("./data/denver_03_29_2021/calendar.csv")
calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,177,2021-03-30,f,$50.00,$50.00,1.0,1125.0
1,3280195,2021-03-29,f,"$1,000.00","$1,000.00",3.0,1125.0
2,3280195,2021-03-30,f,"$1,000.00","$1,000.00",3.0,1125.0
3,3280195,2021-03-31,f,"$1,000.00","$1,000.00",3.0,1125.0
4,3280195,2021-04-01,f,"$1,000.00","$1,000.00",3.0,1125.0


In [4]:
# reviews.csv
reviews = pd.read_csv("./data/denver_03_29_2021/reviews.csv")
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,177,99461681,2016-09-04,78348548,Brenna,"Great place to stay! Definitely a tiny home, b..."
1,177,101460574,2016-09-13,72125554,Rachael,Joe was very friendly and gave us all the info...
2,177,101681475,2016-09-14,21300329,Haley,Amazing property and Joe was a great host! Eve...
3,177,102963808,2016-09-19,53882399,Mike,"Joe was a great host, the description of this ..."
4,177,103447333,2016-09-22,94621370,Carol,He welcomes us when we first arrived and gave ...


In [5]:
# neighbourhoods.csv
neighborhoods = pd.read_csv("./data/denver_03_29_2021/neighbourhoods.csv")
neighborhoods.head()

,neighbourhood_group,neighbourhood
0,NaN,Athmar Park
1,NaN,Auraria
2,NaN,Baker
3,NaN,Barnum
4,NaN,Barnum West


In [6]:
neighborhoods_geojson = gpd.read_file("./data/denver_03_29_2021/neighbourhoods.geojson")
neighborhoods_geojson.head()

,neighbourhood,neighbourhood_group,geometry
0,Chaffee Park,None,"MULTIPOLYGON (((-104.99812 39.79102, -104.9981..."
1,Sunnyside,None,"MULTIPOLYGON (((-104.99818 39.78256, -104.9982..."
2,Highland,None,"MULTIPOLYGON (((-104.99820 39.76930, -104.9982..."
3,University,None,"MULTIPOLYGON (((-104.95945 39.68473, -104.9594..."
4,Globeville,None,"MULTIPOLYGON (((-104.96875 39.79104, -104.9688..."


## Data Cleaning

In [7]:
# listings.csv

# remove unneeded columns
listings = listings.drop(["picture_url", "host_url", "host_thumbnail_url", "host_picture_url", "license"], axis=1)

# TODO: replace all empty columns with NaN

# change columns with 't' & 'f' to booleans
def convert_to_bool(x):
    if x == "t":
        return True
    elif x == "f":
        return False
    else:
        return np.NaN

listings["host_is_superhost"] = listings["host_is_superhost"].apply(lambda x: convert_to_bool(x))
listings["host_identity_verified"] = listings["host_identity_verified"].apply(lambda x: convert_to_bool(x))
listings["has_availability"] = listings["has_availability"].apply(lambda x: convert_to_bool(x))
listings["instant_bookable"] = listings["instant_bookable"].apply(lambda x: convert_to_bool(x))

# Clean up the "bathrooms_text" column to make two new columns - number of bathrooms, and a boolean column to indicate if bthrooms are private
def get_bathrooms_num(x):
    if type(x) != str:
        return 0.0
    elif x == "Shared half-bath" or x == "Half-bath":
        return 0.5
    elif "shared bath" in x:
        return float(x.strip("s").strip("shared bath"))
    elif "private bath" in x:
        return float(x.strip("s").strip("private bath"))
    elif "bath" in x:
        return float(x.strip("s").strip("bath"))
    else:
        return 0.0
    
def get_is_private_bathroom(x):
    if type(x) != str:
        return np.NaN
    elif "shared" in x:
        return False
    else:
        return True

listings["bathrooms"] = listings["bathrooms_text"].apply(lambda x: get_bathrooms_num(x))
listings["private_bathroom"] = listings["bathrooms_text"].apply(lambda x: get_is_private_bathroom(x))
listings = listings.drop("bathrooms_text", axis=1)

# display head to show success
listings.head()


,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,host_id,host_name,host_since,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,private_bathroom
0,177,https://www.airbnb.com/rooms/177,20210329150448,2021-03-30,Tiny Home in the Heart of the City- ECO FRIENDLY,"160 sq ft + 80 sq ft loft for sleeping, Sleeps...","Quiet neighborhood next to park, creeks and bi...",615,Joe,2008-07-07,...,10.0,10.0,9.0,True,2,2,0,0,1.40,True
1,360,https://www.airbnb.com/rooms/360,20210329150448,2021-03-30,Sit in the Peaceful Garden of the Chickadee Co...,Enjoy the famous Colorado weather and unplug i...,The cottage is located in the center of Lower ...,666,Jennifer & Giovanni,2008-07-08,...,10.0,10.0,10.0,False,2,2,0,0,4.34,True
2,364,https://www.airbnb.com/rooms/364,20210329150448,2021-03-31,Lodo / RiNo LOFT via airport train,"Modern 1,000 square foot loft in the heart of ...","Ten brewpubs within walking distance, two grea...",783,Jason,2008-07-11,...,10.0,9.0,9.0,False,1,1,0,0,0.60,True
3,590,https://www.airbnb.com/rooms/590,20210329150448,2021-04-02,Comfortable - and a great value!,"Large guest room in my home, where I also live...",I love the diversity of my neighborhood and it...,933,Jill,2008-07-21,...,10.0,10.0,10.0,False,2,0,2,0,4.14,False
4,592,https://www.airbnb.com/rooms/592,20210329150448,2021-03-31,private,This room is in the basement. It does not hav...,NaN,933,Jill,2008-07-21,...,10.0,10.0,10.0,False,2,0,2,0,1.06,False
